<a href="https://colab.research.google.com/github/MIT-LCP/2019_tokyo_datathon/blob/master/02_severity_of_illness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# eICU Collaborative Research Database

# Notebook 2: Severity of illness

This notebook introduces high level admission details relating to a single patient stay, using the following tables:

ここでは、以下のテーブルを用いて患者のより詳細な入院情報を抽出します。

- patient
- admissiondx
- apacheapsvar
- apachepredvar
- apachepatientresult



## Load libraries and connect to the database

Notebook 1と同様、必要なlibraryとデータベースへのアクセスを行ってください。

In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
# authenticate
auth.authenticate_user()

In [ ]:
# Set up environment variables
project_id='datathonjapan2019'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

To make our lives easier, we'll also install and import a set of helper functions from the `datathon2` package.



In [ ]:
!pip install datathon2

In [ ]:
import datathon2 as dtn

## Selecting a single patient stay¶

As we have seen, the patient table includes general information about the patient admissions (for example, demographics, admission and discharge details). See: http://eicu-crd.mit.edu/eicutables/patient/

まずは、`patient`テーブルからのqueryを行ってみます。

## Questions

Use your knowledge from the previous notebook and the online documentation (http://eicu-crd.mit.edu/) to answer the following questions:

- Which column in the patient table is distinct for each stay in the ICU (similar to `icustay_id` in MIMIC-III)?
- Which column is unique for each patient (similar to `subject_id` in MIMIC-III)?

注：MIMIC-IIIでは、患者一人一人が異なるIDを持っていますが（`subject_id`）、同人物が複数回ICUに入室した場合それぞれ異なる`icustay_id`が割り振られます。

In [ ]:
# view distinct ids
query = """
SELECT DISTINCT(patientunitstayid)
FROM `physionet-data.eicu_crd_demo.patient`
"""

dtn.run_query(query,project_id)

In [ ]:
# select a single ICU stay（適当に一つIDを選んでください）
patientunitstayid = <your_id_here>

In [ ]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.patient`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

patient = dtn.run_query(query,project_id)

In [ ]:
patient

## Questions

- Which type of unit was the patient admitted to? Hint: Try `patient['unittype']` or `patient.unittype`（どこへ入室したか）
- What year was the patient discharged from the ICU? Hint: You can view the table columns with `patient.columns`（いつ退室したか）
- What was the status of the patient upon discharge from the unit?（退室時の状態は？）

## The admissiondx table

The `admissiondx` table contains the primary diagnosis for admission to the ICU according to the APACHE scoring criteria. For more detail, see: http://eicu-crd.mit.edu/eicutables/admissiondx/

`admissiondx`テーブルからは、ICU入室時の診断に関する情報が抽出できます。

In [ ]:
# set the where clause to select the stay of interest
# `WHERE`で条件の絞り込み。
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.admissiondx`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

admissiondx = dtn.run_query(query,project_id)

In [ ]:
# View the columns in this data
admissiondx.columns

In [ ]:
# View the data
admissiondx.head()

In [ ]:
# Set the display options to avoid truncating the text
# `admitdxpath`の表示方法を変えます。
pd.set_option('display.max_colwidth', -1)
admissiondx.admitdxpath

## Questions

- What was the primary reason for admission?（入室理由は？）
- How soon after admission to the ICU was the diagnoses recorded in eCareManager? Hint: The `offset` columns indicate the time in minutes after admission to the ICU. （入室後いつ診断されたか？）

## The apacheapsvar table

The apacheapsvar table contains the variables used to calculate the Acute Physiology Score (APS) III for patients. APS-III is an established method of summarizing patient severity of illness on admission to the ICU, taking the "worst" observations for a patient in a 24 hour period.

次に、APS-III（重症度スコア）に関するデータを抽出するため、`apacheapsvar`テーブルを参照します。

The score is part of the Acute Physiology Age Chronic Health Evaluation (APACHE) system of equations for predicting outcomes for ICU patients. See: http://eicu-crd.mit.edu/eicutables/apacheApsVar/

In [ ]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apacheapsvar`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apacheapsvar = dtn.run_query(query,project_id)

In [ ]:
apacheapsvar.head()

## Questions

- What was the 'worst' heart rate recorded for the patient during the scoring period?（重症度スコアリング中の`最悪`の心拍数は？）
- Was the patient oriented and able to converse normally on the day of admission? (hint: the verbal element refers to the Glasgow Coma Scale).（見当識障害は？）

# apachepredvar table

The apachepredvar table provides variables underlying the APACHE predictions. Acute Physiology Age Chronic Health Evaluation (APACHE) consists of a groups of equations used for predicting outcomes in critically ill patients. See: http://eicu-crd.mit.edu/eicutables/apachePredVar/

このテーブルは、APACHEスコアについての元情報も含んでいます。

In [ ]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apachepredvar`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apachepredvar = dtn.run_query(query,project_id)

In [ ]:
apachepredvar.columns

## Questions

- Was the patient ventilated during (APACHE) day 1 of their stay?（APACHEスコアリング1日目の機械換気？）
- Is the patient recorded as having diabetes?（糖尿病の既往？）

# `apachepatientresult` table

The `apachepatientresult` table provides predictions made by the APACHE score (versions IV and IVa), including probability of mortality, length of stay, and ventilation days. See: http://eicu-crd.mit.edu/eicutables/apachePatientResult/

`apachepatientresult`テーブルでは、APACHEスコアを用いた死亡率やICU滞在日数、挿管日数に関する予測値をみることができます。

In [ ]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apachepatientresult`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apachepatientresult = dtn.run_query(query,project_id)

In [ ]:
apachepatientresult

## Questions

- What versions of the APACHE score are computed?（APACHEスコアのバージョンは？）
- How many days during the stay was the patient ventilated?（挿管日数は？）
- How long was the patient predicted to stay in hospital?（病院滞在日数は？）
- Was this prediction close to the truth?（正確性？）